# Gesture Recognition
In this group project, you are going to build a 3D Conv model that will be able to predict the 5 gestures correctly. Please import the following libraries to get started.

In [19]:
import numpy as np
import os
from scipy.misc import imread, imresize
import datetime
import os
import scipy

We set the random seed so that the results don't vary drastically.

In [20]:
np.random.seed(30)
import random as rn
rn.seed(30)
from keras import backend as K
import tensorflow as tf
tf.set_random_seed(30)

In this block, you read the folder names for training and validation. You also set the `batch_size` here. Note that you set the batch size in such a way that you are able to use the GPU in full capacity. You keep increasing the batch size until the machine throws an error.

##### <font color='red'> Path to train and test csv's => how to submit?</font>

In [21]:
# train_doc = np.random.permutation(open('/notebooks/storage/Final_data/Collated_training/train.csv').readlines())
# val_doc = np.random.permutation(open('/notebooks/storage/Final_data/Collated_training/val.csv').readlines())

train_doc = np.random.permutation(open('./Project_data/train.csv').readlines())
val_doc = np.random.permutation(open('./Project_data/val.csv').readlines())


## Generator
This is one of the most important part of the code. The overall structure of the generator has been given. In the generator, you are going to preprocess the images as you have images of 2 different dimensions as well as create a batch of video frames. You have to experiment with `img_idx`, `y`,`z` and normalization such that you get high accuracy.

In [22]:
def norm1(image):
    epsilon = 1e-7
    norm_image = (image - image.mean(axis=(0,1,2), keepdims=True)) / image.std(axis=(0,1,2), keepdims=True)
    return norm_image

In [23]:
def norm2(image):
    return np.array((image - np.min(image)) / (np.max(image) - np.min(image)))

In [24]:
def norm3(image):
    return (image - np.percentile(image,5)) / (np.percentile(image,95) - np.percentile(image,5))

In [25]:
def generator(source_path, folder_list, batch_size, image_idx,img_dim,image_norm_method=1 ):
    print( 'Source path = ', source_path, '; batch size =', batch_size)
    # img_idx = #create a list of image numbers you want to use for a particular video
    img_idx = image_idx
    while True:
        t = np.random.permutation(folder_list)
        num_batches = len(folder_list) // batch_size
        
        # for remaining videos
        remaining_sequences = len(folder_list) % batch_size
#         print("batch_size = " , batch_size, " and sequences = " , remaining_sequences)
        x = len(img_idx)
        y = img_dim[0]
        z = img_dim[1]        
        
        for batch in range(num_batches): # we iterate over the number of batches   
#             print("current batch = ", batch)
            batch_data = np.zeros((batch_size,x,y,z,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
            
            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    
                    if(image.shape[0]!=image.shape[1]):
                        image=image[0:120,20:140]
                    
                    # `imresize` is deprecated!
                    # `imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
                    # Use ``skimage.transform.resize`` instead.
                    image = imresize(image, (y,z))
                    #crop the images and resize them. Note that the images are of 2 different shape 
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                    
                    # after normalization, the values will be converted to float, is that fine?
                    if image_norm_method==1:
                        norm_image = norm1(image)
                    elif image_norm_method ==2:
                        norm_image = norm2(image)
                    else:
                        norm_image = norm3(image)
                    
                    batch_data[folder,idx,:,:,0] = norm_image[:,:,0]
                    batch_data[folder,idx,:,:,1] = norm_image[:,:,1]
                    batch_data[folder,idx,:,:,2] = norm_image[:,:,2]
                    
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do

        
        # write the code for the remaining data points which are left after full batches
#         print("processing remaining sequences = " ,remaining_sequences, "::num_batches=", num_batches)
        if remaining_sequences != 0:
            batch_size = remaining_sequences
            batch_data = np.zeros((batch_size,x,y,z,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output

            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (num_batches*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = imread(source_path+'/'+ t[folder + (num_batches*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    
                    if(image.shape[0]!=image.shape[1]):
                        image=image[0:120,20:140]
                    image = imresize(image, (y,z))
                    
                    if image_norm_method==1:
                        norm_image = norm1(image)
                    else:
                        norm_image = norm2(image)

                    batch_data[folder,idx,:,:,0] = norm_image[:,:,0]
                    batch_data[folder,idx,:,:,1] = norm_image[:,:,1]
                    batch_data[folder,idx,:,:,2] = norm_image[:,:,2]

                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
                
                yield batch_data, batch_labels


Note here that a video is represented above in the generator as (number of images, height, width, number of channels). Take this into consideration while creating the model architecture.

In [26]:
curr_dt_time = datetime.datetime.now()
# train_path = '/notebooks/storage/Final_data/Collated_training/train'
# val_path = '/notebooks/storage/Final_data/Collated_training/val'
num_train_sequences = len(train_doc)
print('# training sequences =', num_train_sequences)
num_val_sequences = len(val_doc)
print('# validation sequences =', num_val_sequences)


# training sequences = 663
# validation sequences = 100


## Model
Here you make the model using different functionalities that Keras provides. Remember to use `Conv3D` and `MaxPooling3D` and not `Conv2D` and `Maxpooling2D` for a 3D convolution model. You would want to use `TimeDistributed` while building a Conv2D + RNN model. Also remember that the last layer is the softmax. Design the network in such a way that the model is able to give good accuracy on the least number of parameters so that it can fit in the memory of the webcam.

In [27]:
from keras.models import Sequential, Model
from keras.layers import Dense, GRU, Flatten, TimeDistributed, Flatten, BatchNormalization, Activation, Dropout
from keras.layers.convolutional import Conv3D, MaxPooling3D
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping
from keras import optimizers
from keras.optimizers import Adam


#write your model here

## 3d CNN models

- https://ieeexplore.ieee.org/document/8545718 (later experiment with more conv)

In [28]:
def create_model(input_shape, filters, optimiser=Adam()):

    filter_size = filters
    # filter_size = [8,16,32,64,128,256]
    dense_layer_size = [256,128] #256,256

#     input_shape=(len(image_idx),image_width,image_height,3)

    model = Sequential()


    #Layer 1
    model.add(Conv3D(filters=filter_size[0],kernel_size=(3,3,3),input_shape=input_shape, padding='same')) # 'valid'
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(MaxPooling3D(pool_size=(2,2,2)))


    #Layer 2
    model.add(Conv3D(filters=filter_size[1],kernel_size=(3,3,3), padding='same')) # 'valid'
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(MaxPooling3D(pool_size=(2,2,2)))


    #Layer 3
    model.add(Conv3D(filters=filter_size[2],kernel_size=(1,3,3), padding='same')) # 'valid'
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(MaxPooling3D(pool_size=(2,2,2)))

    # model.add(Dropout(0.25))

    # #Layer 4
    model.add(Conv3D(filters=filter_size[3],kernel_size=(1,3,3), padding='same')) # 'valid'
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(MaxPooling3D(pool_size=(2,2,2)))

    model.add(Dropout(0.25))


    #Flatten Layers
    model.add(Flatten())

    # Dense 1
    model.add(Dense(dense_layer_size[0], activation='relu'))
    model.add(Dropout(0.5))

    # Dense 2
    model.add(Dense(dense_layer_size[1], activation='relu'))
    model.add(Dropout(0.5))

    # final softmax
    #softmax layer
    model.add(Dense(5, activation='softmax'))
    
    model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
    print (model.summary())
    
    return model

### Convolutions + RNN
- The conv2D network will extract a feature vector for each image, and a sequence of these feature vectors is then fed to an RNN-based network. The output of the RNN is a regular softmax (for a classification problem such as this one).

#### There are a few key things to note about the conv-RNN architecture:

- You can use transfer learning in the 2D CNN layer rather than training your own CNN 
- GRU can be a better choice than an LSTM since it has lesser number of gates (and thus parameters)

Now that you have written the model, the next step is to `compile` the model. When you print the `summary` of the model, you'll see the total number of parameters you have to train.

In [29]:
# optimiser = Adam()
# model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
# print (model.summary())

### <font color='green'>TODO - select the list of image indexes from the video
- take +/-7 images from the mid position </font>

### <font color='green'>TODO - num_batches formula 
    
- x = len(img_idx)
- y = height of image
- z = width of image

In [30]:
# image_width = 100
# image_height = 100
# # image_idx = list(np.arange(0,30,2))
# image_idx = [0,1,2,4,6,8,10,12,14,16,18,20,22,24,26,27,28,29]
# print(len(image_idx))

# batch_size = 8

# num_epochs = 20 #50
# print ('# epochs =', num_epochs)

Let us create the `train_generator` and the `val_generator` which will be used in `.fit_generator`.

In [31]:
def create_train_val_generators(batch_size, image_idx,img_dim, image_norm_method=1):
    train_path = './Project_data/train'
    val_path = './Project_data/val'

    train_generator = generator(train_path, train_doc, batch_size, image_idx,img_dim,image_norm_method=image_norm_method)
    val_generator = generator(val_path, val_doc, batch_size, image_idx, img_dim,image_norm_method=image_norm_method)
    
    return train_generator, val_generator

The `steps_per_epoch` and `validation_steps` are used by `fit_generator` to decide the number of next() calls it need to make.

In [32]:
def train_int_model(model, train_generator, val_generator,batch_size, num_epochs, model_log):
    if (num_train_sequences%batch_size) == 0:
        steps_per_epoch = int(num_train_sequences/batch_size)
    else:
        steps_per_epoch = (num_train_sequences//batch_size) + 1

    if (num_val_sequences%batch_size) == 0:
        validation_steps = int(num_val_sequences/batch_size)
    else:
        validation_steps = (num_val_sequences//batch_size) + 1
        
    model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + model_log +'/'
    
    if not os.path.exists(model_name):
        os.mkdir(model_name)

    filepath = model_name + 'model_-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'
    #early stopping required
    ES = EarlyStopping(monitor='val_loss', verbose=1, patience=5)
    checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, 
                                 save_weights_only=False, mode='auto', period=1)
    # patience can be increased with early stopping (5)
    LR = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, cooldown=1, verbose=1)
    # callbacks_list = [checkpoint, LR, ES]
    callbacks_list = [checkpoint, LR]
    
    
    model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

In [37]:
def train_model(epoch, batch_size, img_norm_method, log, optimizer=Adam(), filters = [16,32,64,128]):
    
    image_width = 100
    image_height = 100
    
    image_idx = [0,1,2,4,6,8,10,12,14,16,18,20,22,24,26,27,28,29]

    batch_size = batch_size
    num_epochs = epoch
    img_dim = (image_height,image_width)
    train_gen, val_gen = create_train_val_generators(batch_size=batch_size,image_idx=image_idx,img_dim=img_dim,image_norm_method=img_norm_method)
    
    # length of video sequence, width, height, num_channels
    input_shape=(len(image_idx), image_width, image_height, 3)
    
    model = create_model(input_shape=input_shape, filters=filters, optimiser=optimizer)
    
    train_int_model(model = model, train_generator=train_gen, val_generator=val_gen, batch_size=batch_size,num_epochs=epoch,model_log=log)

### Effect of Normalization

In [38]:
print("1"*100)
train_model(epoch=20, batch_size=8, img_norm_method=1, log="norm_1",filters = [16,32,64,128])
print("1"*100)

1111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_9 (Conv3D)            (None, 18, 100, 100, 16)  1312      
_________________________________________________________________
batch_normalization_9 (Batch (None, 18, 100, 100, 16)  64        
_________________________________________________________________
activation_9 (Activation)    (None, 18, 100, 100, 16)  0         
_________________________________________________________________
max_pooling3d_9 (MaxPooling3 (None, 9, 50, 50, 16)     0         
_________________________________________________________________
conv3d_10 (Conv3D)           (None, 9, 50, 50, 32)     13856     
_________________________________________________________________
batch_normalization_10 (Batc (None, 9, 50, 50, 32)     128       
_________________________________________

/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:24: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:32: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


82/83 [============================>.] - ETA: 1s - loss: 3.8308 - categorical_accuracy: 0.2180

/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:62: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:66: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


83/83 [==============================] - 136s 2s/step - loss: 3.7883 - categorical_accuracy: 0.2171 - val_loss: 1.4953 - val_categorical_accuracy: 0.2300

Epoch 00001: saving model to model_init_2019-09-0916_24_45.232582norm_1/model_-00001-3.79349-0.21719-1.49534-0.23000.h5
Epoch 2/20
83/83 [==============================] - 38s 457ms/step - loss: 1.6470 - categorical_accuracy: 0.2134 - val_loss: 1.6122 - val_categorical_accuracy: 0.2500

Epoch 00002: saving model to model_init_2019-09-0916_24_45.232582norm_1/model_-00002-1.64701-0.21343-1.61220-0.25000.h5
Epoch 3/20
83/83 [==============================] - 32s 385ms/step - loss: 1.6113 - categorical_accuracy: 0.2506 - val_loss: 1.5254 - val_categorical_accuracy: 0.3654

Epoch 00003: saving model to model_init_2019-09-0916_24_45.232582norm_1/model_-00003-1.61386-0.24944-1.52540-0.36538.h5
Epoch 4/20
83/83 [==============================] - 29s 345ms/step - loss: 1.6011 - categorical_accuracy: 0.1927 - val_loss: 1.6073 - val_categorical

In [39]:
print("2"*100)
train_model(epoch=20, batch_size=8, img_norm_method=2,log="norm_2",filters = [16,32,64,128])
print("2"*100)

2222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_13 (Conv3D)           (None, 18, 100, 100, 16)  1312      
_________________________________________________________________
batch_normalization_13 (Batc (None, 18, 100, 100, 16)  64        
_________________________________________________________________
activation_13 (Activation)   (None, 18, 100, 100, 16)  0         
_________________________________________________________________
max_pooling3d_13 (MaxPooling (None, 9, 50, 50, 16)     0         
_________________________________________________________________
conv3d_14 (Conv3D)           (None, 9, 50, 50, 32)     13856     
_________________________________________________________________
batch_normalization_14 (Batc (None, 9, 50, 50, 32)     128       
_________________________________________

/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:24: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:32: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


82/83 [============================>.] - ETA: 0s - loss: 7.4043 - categorical_accuracy: 0.2439

/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:62: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:66: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


83/83 [==============================] - 49s 593ms/step - loss: 7.3152 - categorical_accuracy: 0.2461 - val_loss: 10.4916 - val_categorical_accuracy: 0.2000

Epoch 00001: saving model to model_init_2019-09-0916_24_45.232582norm_2/model_-00001-7.32613-0.24585-10.49157-0.20000.h5
Epoch 2/20
83/83 [==============================] - 37s 443ms/step - loss: 2.5560 - categorical_accuracy: 0.2651 - val_loss: 2.6808 - val_categorical_accuracy: 0.3846

Epoch 00002: saving model to model_init_2019-09-0916_24_45.232582norm_2/model_-00002-2.55598-0.26506-2.68083-0.38462.h5
Epoch 3/20
83/83 [==============================] - 30s 361ms/step - loss: 1.6562 - categorical_accuracy: 0.2572 - val_loss: 1.4939 - val_categorical_accuracy: 0.3077

Epoch 00003: saving model to model_init_2019-09-0916_24_45.232582norm_2/model_-00003-1.65356-0.26058-1.49386-0.30769.h5
Epoch 4/20
83/83 [==============================] - 27s 322ms/step - loss: 1.5890 - categorical_accuracy: 0.2578 - val_loss: 1.4884 - val_categor

In [40]:
print("3"*100)
train_model(epoch=20, batch_size=8, img_norm_method=3,log="norm_3",filters = [16,32,64,128])
print("3"*100)

3333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_17 (Conv3D)           (None, 18, 100, 100, 16)  1312      
_________________________________________________________________
batch_normalization_17 (Batc (None, 18, 100, 100, 16)  64        
_________________________________________________________________
activation_17 (Activation)   (None, 18, 100, 100, 16)  0         
_________________________________________________________________
max_pooling3d_17 (MaxPooling (None, 9, 50, 50, 16)     0         
_________________________________________________________________
conv3d_18 (Conv3D)           (None, 9, 50, 50, 32)     13856     
_________________________________________________________________
batch_normalization_18 (Batc (None, 9, 50, 50, 32)     128       
_________________________________________

/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:24: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:32: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


82/83 [============================>.] - ETA: 0s - loss: 3.4021 - categorical_accuracy: 0.2210

/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:62: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:66: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


83/83 [==============================] - 56s 674ms/step - loss: 3.3651 - categorical_accuracy: 0.2184 - val_loss: 1.5356 - val_categorical_accuracy: 0.3000

Epoch 00001: saving model to model_init_2019-09-0916_24_45.232582norm_3/model_-00001-3.36962-0.21870-1.53555-0.30000.h5
Epoch 2/20
83/83 [==============================] - 42s 506ms/step - loss: 1.6121 - categorical_accuracy: 0.2599 - val_loss: 1.5931 - val_categorical_accuracy: 0.2308

Epoch 00002: saving model to model_init_2019-09-0916_24_45.232582norm_3/model_-00002-1.61208-0.25990-1.59312-0.23077.h5
Epoch 3/20
83/83 [==============================] - 35s 418ms/step - loss: 1.6139 - categorical_accuracy: 0.2018 - val_loss: 1.6046 - val_categorical_accuracy: 0.2500

Epoch 00003: saving model to model_init_2019-09-0916_24_45.232582norm_3/model_-00003-1.61223-0.20490-1.60461-0.25000.h5
Epoch 4/20
83/83 [==============================] - 31s 373ms/step - loss: 1.5637 - categorical_accuracy: 0.2867 - val_loss: 1.6069 - val_categoric

- After checking which is normalization method check the batch size

### Effect of batch size

In [41]:
print("4"*100)
train_model(epoch=20, batch_size=8, img_norm_method=2, log="batch_8",filters = [16,32,64,128])
print("4"*100)

4444444444444444444444444444444444444444444444444444444444444444444444444444444444444444444444444444
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_21 (Conv3D)           (None, 18, 100, 100, 16)  1312      
_________________________________________________________________
batch_normalization_21 (Batc (None, 18, 100, 100, 16)  64        
_________________________________________________________________
activation_21 (Activation)   (None, 18, 100, 100, 16)  0         
_________________________________________________________________
max_pooling3d_21 (MaxPooling (None, 9, 50, 50, 16)     0         
_________________________________________________________________
conv3d_22 (Conv3D)           (None, 9, 50, 50, 32)     13856     
_________________________________________________________________
batch_normalization_22 (Batc (None, 9, 50, 50, 32)     128       
_________________________________________

/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:24: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:32: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


82/83 [============================>.] - ETA: 0s - loss: 2.7323 - categorical_accuracy: 0.2027

/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:62: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:66: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


83/83 [==============================] - 48s 581ms/step - loss: 2.7046 - categorical_accuracy: 0.2020 - val_loss: 1.3577 - val_categorical_accuracy: 0.3700

Epoch 00001: saving model to model_init_2019-09-0916_24_45.232582batch_8/model_-00001-2.70795-0.20211-1.35767-0.37000.h5
Epoch 2/20
83/83 [==============================] - 35s 425ms/step - loss: 1.6793 - categorical_accuracy: 0.2599 - val_loss: 1.5387 - val_categorical_accuracy: 0.3077

Epoch 00002: saving model to model_init_2019-09-0916_24_45.232582batch_8/model_-00002-1.67932-0.25990-1.53867-0.30769.h5
Epoch 3/20
83/83 [==============================] - 29s 347ms/step - loss: 1.4853 - categorical_accuracy: 0.3118 - val_loss: 1.3911 - val_categorical_accuracy: 0.4038

Epoch 00003: saving model to model_init_2019-09-0916_24_45.232582batch_8/model_-00003-1.49174-0.30735-1.39113-0.40385.h5
Epoch 4/20
83/83 [==============================] - 26s 312ms/step - loss: 1.6348 - categorical_accuracy: 0.2771 - val_loss: 1.4387 - val_catego

In [42]:
print("2"*100)
train_model(epoch=20, batch_size=16, img_norm_method=2,log="batch_16",filters = [16,32,64,128])
print("2"*100)

2222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_25 (Conv3D)           (None, 18, 100, 100, 16)  1312      
_________________________________________________________________
batch_normalization_25 (Batc (None, 18, 100, 100, 16)  64        
_________________________________________________________________
activation_25 (Activation)   (None, 18, 100, 100, 16)  0         
_________________________________________________________________
max_pooling3d_25 (MaxPooling (None, 9, 50, 50, 16)     0         
_________________________________________________________________
conv3d_26 (Conv3D)           (None, 9, 50, 50, 32)     13856     
_________________________________________________________________
batch_normalization_26 (Batc (None, 9, 50, 50, 32)     128       
_________________________________________

/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:24: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:32: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


 6/42 [===>..........................] - ETA: 52s - loss: 3.1164 - categorical_accuracy: 0.1979

/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:62: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:66: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


42/42 [==============================] - 46s 1s/step - loss: 2.4277 - categorical_accuracy: 0.2374 - val_loss: 1.4290 - val_categorical_accuracy: 0.3900

Epoch 00001: saving model to model_init_2019-09-0916_24_45.232582batch_16/model_-00001-2.44209-0.23680-1.42895-0.39000.h5
Epoch 2/20
42/42 [==============================] - 17s 402ms/step - loss: 1.8130 - categorical_accuracy: 0.2483 - val_loss: 1.5950 - val_categorical_accuracy: 0.2143

Epoch 00002: saving model to model_init_2019-09-0916_24_45.232582batch_16/model_-00002-1.81301-0.24830-1.59496-0.21429.h5
Epoch 3/20
42/42 [==============================] - 19s 447ms/step - loss: 1.7254 - categorical_accuracy: 0.2483 - val_loss: 1.3588 - val_categorical_accuracy: 0.4286

Epoch 00003: saving model to model_init_2019-09-0916_24_45.232582batch_16/model_-00003-1.72536-0.24830-1.35881-0.42857.h5
Epoch 4/20
42/42 [==============================] - 16s 377ms/step - loss: 1.5103 - categorical_accuracy: 0.2695 - val_loss: 1.4788 - val_catego

In [43]:
print("3"*100)
train_model(epoch=20, batch_size=32, img_norm_method=2,log="batch_32",filters = [16,32,64,128])
print("3"*100)

3333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_29 (Conv3D)           (None, 18, 100, 100, 16)  1312      
_________________________________________________________________
batch_normalization_29 (Batc (None, 18, 100, 100, 16)  64        
_________________________________________________________________
activation_29 (Activation)   (None, 18, 100, 100, 16)  0         
_________________________________________________________________
max_pooling3d_29 (MaxPooling (None, 9, 50, 50, 16)     0         
_________________________________________________________________
conv3d_30 (Conv3D)           (None, 9, 50, 50, 32)     13856     
_________________________________________________________________
batch_normalization_30 (Batc (None, 9, 50, 50, 32)     128       
_________________________________________

/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:24: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:32: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


 3/21 [===>..........................] - ETA: 56s - loss: 3.5408 - categorical_accuracy: 0.1875 

/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:62: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:66: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


21/21 [==============================] - 48s 2s/step - loss: 2.7956 - categorical_accuracy: 0.2159 - val_loss: 1.5388 - val_categorical_accuracy: 0.3100

Epoch 00001: saving model to model_init_2019-09-0916_24_45.232582batch_32/model_-00001-2.81711-0.21870-1.53876-0.31000.h5
Epoch 2/20
21/21 [==============================] - 10s 460ms/step - loss: 1.6818 - categorical_accuracy: 0.2340 - val_loss: 1.5990 - val_categorical_accuracy: 0.4375

Epoch 00002: saving model to model_init_2019-09-0916_24_45.232582batch_32/model_-00002-1.68181-0.23395-1.59899-0.43750.h5
Epoch 3/20
21/21 [==============================] - 28s 1s/step - loss: 1.7919 - categorical_accuracy: 0.2754 - val_loss: 1.6102 - val_categorical_accuracy: 0.1250

Epoch 00003: saving model to model_init_2019-09-0916_24_45.232582batch_32/model_-00003-1.79193-0.27536-1.61024-0.12500.h5
Epoch 4/20
21/21 [==============================] - 18s 836ms/step - loss: 1.5849 - categorical_accuracy: 0.2011 - val_loss: 1.5056 - val_categoric

### Effect of filter size

In [44]:
print("4"*100)
train_model(epoch=20, batch_size=8, img_norm_method=2, log="filter_8",filters=[8,16,32,64])
print("4"*100)

4444444444444444444444444444444444444444444444444444444444444444444444444444444444444444444444444444
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_33 (Conv3D)           (None, 18, 100, 100, 8)   656       
_________________________________________________________________
batch_normalization_33 (Batc (None, 18, 100, 100, 8)   32        
_________________________________________________________________
activation_33 (Activation)   (None, 18, 100, 100, 8)   0         
_________________________________________________________________
max_pooling3d_33 (MaxPooling (None, 9, 50, 50, 8)      0         
_________________________________________________________________
conv3d_34 (Conv3D)           (None, 9, 50, 50, 16)     3472      
_________________________________________________________________
batch_normalization_34 (Batc (None, 9, 50, 50, 16)     64        
_________________________________________

/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:24: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:32: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


82/83 [============================>.] - ETA: 0s - loss: 3.4421 - categorical_accuracy: 0.1677

/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:62: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:66: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


83/83 [==============================] - 49s 591ms/step - loss: 3.4161 - categorical_accuracy: 0.1674 - val_loss: 1.7400 - val_categorical_accuracy: 0.2600

Epoch 00001: saving model to model_init_2019-09-0916_24_45.232582filter_8/model_-00001-3.41930-0.16742-1.74003-0.26000.h5
Epoch 2/20
83/83 [==============================] - 35s 425ms/step - loss: 2.7837 - categorical_accuracy: 0.1842 - val_loss: 1.5566 - val_categorical_accuracy: 0.3846

Epoch 00002: saving model to model_init_2019-09-0916_24_45.232582filter_8/model_-00002-2.78374-0.18417-1.55664-0.38462.h5
Epoch 3/20
83/83 [==============================] - 30s 362ms/step - loss: 2.6757 - categorical_accuracy: 0.2062 - val_loss: 1.6435 - val_categorical_accuracy: 0.2308

Epoch 00003: saving model to model_init_2019-09-0916_24_45.232582filter_8/model_-00003-2.67434-0.20713-1.64345-0.23077.h5
Epoch 4/20
83/83 [==============================] - 26s 312ms/step - loss: 2.4929 - categorical_accuracy: 0.1847 - val_loss: 1.6994 - val_cat

In [45]:
print("5"*100)
train_model(epoch=20, batch_size=8, img_norm_method=2, log="filter_16",filters=[16,32,64,128])
print("5"*100)

5555555555555555555555555555555555555555555555555555555555555555555555555555555555555555555555555555
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_37 (Conv3D)           (None, 18, 100, 100, 16)  1312      
_________________________________________________________________
batch_normalization_37 (Batc (None, 18, 100, 100, 16)  64        
_________________________________________________________________
activation_37 (Activation)   (None, 18, 100, 100, 16)  0         
_________________________________________________________________
max_pooling3d_37 (MaxPooling (None, 9, 50, 50, 16)     0         
_________________________________________________________________
conv3d_38 (Conv3D)           (None, 9, 50, 50, 32)     13856     
_________________________________________________________________
batch_normalization_38 (Batc (None, 9, 50, 50, 32)     128       
_________________________________________

/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:24: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:32: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


82/83 [============================>.] - ETA: 0s - loss: 3.2803 - categorical_accuracy: 0.1845

/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:62: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:66: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


83/83 [==============================] - 47s 564ms/step - loss: 3.2533 - categorical_accuracy: 0.1857 - val_loss: 1.5885 - val_categorical_accuracy: 0.2300

Epoch 00001: saving model to model_init_2019-09-0916_24_45.232582filter_16/model_-00001-3.25658-0.18552-1.58855-0.23000.h5
Epoch 2/20
83/83 [==============================] - 35s 423ms/step - loss: 2.6625 - categorical_accuracy: 0.2186 - val_loss: 1.6697 - val_categorical_accuracy: 0.2308

Epoch 00002: saving model to model_init_2019-09-0916_24_45.232582filter_16/model_-00002-2.66246-0.21859-1.66971-0.23077.h5
Epoch 3/20
83/83 [==============================] - 28s 342ms/step - loss: 2.5314 - categorical_accuracy: 0.2000 - val_loss: 1.5631 - val_categorical_accuracy: 0.2692

Epoch 00003: saving model to model_init_2019-09-0916_24_45.232582filter_16/model_-00003-2.55844-0.19822-1.56309-0.26923.h5
Epoch 4/20
83/83 [==============================] - 27s 330ms/step - loss: 2.5605 - categorical_accuracy: 0.1494 - val_loss: 1.5243 - val_

In [46]:
print("6"*100)
train_model(epoch=20, batch_size=8, img_norm_method=2, log="filter_32",filters=[32,64,128,256])
print("6"*100)

6666666666666666666666666666666666666666666666666666666666666666666666666666666666666666666666666666
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_41 (Conv3D)           (None, 18, 100, 100, 32)  2624      
_________________________________________________________________
batch_normalization_41 (Batc (None, 18, 100, 100, 32)  128       
_________________________________________________________________
activation_41 (Activation)   (None, 18, 100, 100, 32)  0         
_________________________________________________________________
max_pooling3d_41 (MaxPooling (None, 9, 50, 50, 32)     0         
_________________________________________________________________
conv3d_42 (Conv3D)           (None, 9, 50, 50, 64)     55360     
_________________________________________________________________
batch_normalization_42 (Batc (None, 9, 50, 50, 64)     256       
_________________________________________

/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:24: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:32: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


81/83 [============================>.] - ETA: 1s - loss: 3.5267 - categorical_accuracy: 0.2052

/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:62: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:66: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


83/83 [==============================] - 51s 620ms/step - loss: 3.4905 - categorical_accuracy: 0.2068 - val_loss: 1.6405 - val_categorical_accuracy: 0.2300

Epoch 00001: saving model to model_init_2019-09-0916_24_45.232582filter_32/model_-00001-3.49380-0.20664-1.64048-0.23000.h5
Epoch 2/20
83/83 [==============================] - 37s 440ms/step - loss: 2.9577 - categorical_accuracy: 0.2014 - val_loss: 1.5970 - val_categorical_accuracy: 0.2500

Epoch 00002: saving model to model_init_2019-09-0916_24_45.232582filter_32/model_-00002-2.95767-0.20138-1.59696-0.25000.h5
Epoch 3/20
83/83 [==============================] - 29s 353ms/step - loss: 2.6539 - categorical_accuracy: 0.1759 - val_loss: 1.4831 - val_categorical_accuracy: 0.3462

Epoch 00003: saving model to model_init_2019-09-0916_24_45.232582filter_32/model_-00003-2.64970-0.17595-1.48315-0.34615.h5
Epoch 4/20
83/83 [==============================] - 27s 323ms/step - loss: 2.3662 - categorical_accuracy: 0.1919 - val_loss: 1.5133 - val_

In [ ]:
# image_idx = [0,1,2,4,6,8,10,12,14,16,18,20,22,24,26,27,28,29]

# # length of video sequence, width, height, num_channels
# input_shape=(len(image_idx), image_width, image_height, 3)
# create_model(input_shape=input_shape, filters=[8,16,32,64,128,256], optimiser=Adam())